#####When we create a dataframe, what will be the number of patitions at the initial stage

#####spark.conf.get("spark.sql.files.maxPartitionBytes)::128MB

####df.rdd.getNumPartitions

- decide in a way so that all the resources  should be used
- if 9 cores are there, then number of paralleism can not be greater then 9
- suppose we haev 1152 MB file
- requested for 3 executors with 3 cpu cores
- number of  total cores = 9
- number of pattiotns = total file size/maxpartition =  1152/128 = 9 partitions

In [0]:
spark=SparkSession.builder.config("spark.dynamicAllocation.enabled", False). \
    config("spark.executors.cores", 3). \
        config("spark.executors.instances", 3). \`
        config("spark.executor.memory", "2g"). \
        .master("local[1]")
      .appName("SparkByExamples.com")
      .getOrCreate()


#####How to decide number of executors, number of cores and executor memory


#####scenario 1 : 16 modes and each node has 8 cores and 32 GB RAM
#####datset : 1TB

1. An executor is a Spark process responsible for executing tasks on a specific node in the cluster. Each executor is assigned a fixed number of cores and a certain amount of memory. The number of executors determines the level of parallelism at which Spark can process data.
- Having more executors allows for better parallelism and resource utilization.
- Each executor can work on a subset of data independently, which can lead to increased processing speed.
- However, it’s important to strike a balance between the number of executors and the available cluster resources. If the number of executors is too high, it can lead to excessive memory usage and increased overhead due to task scheduling.

2. Spark Cores
The number of cores refers to the total number of processing units available on the machines in your Spark cluster. It represents the parallelism level at which Spark can execute tasks. Each core can handle one concurrent task.

- Increasing the number of cores allows,

- Spark to execute more tasks simultaneously, which can improve the overall throughput of your application.
- However, adding too many cores can also introduce overhead due to task scheduling and inter-node communication, especially if the cluster resources are limited.

3. Configuring Spark Number of Executors and its Cores
Configuring the number of cores and executors in Apache Spark depends on several factors, including

- The characteristics of your workload,
- The available cluster resources, and
Specific requirements of your application.

**Number of executors:
**The number of executors should be equal to the number of cores on each node in the cluster.
If there are more cores than nodes, then the number of executors should be equal to the number of nodes.
**Memory per executor:
**The amount of memory allocated to each executor should be based on the size of the data that will be processed by that executor.
It is important to leave some memory available for the operating system and other processes.
A good starting point is to allocate 1GB of memory per executor.
**Number of partitions:
**The number of partitions used for shuffle operations should be equal to the number of executors.


1. Available Resources:
Total cores in the cluster = 16 nodes * 8 cores per node = 128 cores
Total memory in the cluster = 16 nodes * 32 GB per node = 512 GB
2. Workload Characteristics: 
Large dataset size and complex computations suggest that you need a high level of parallelism to efficiently process the data. Let’s assume that you want to allocate 80% of the available resources to Spark.

####Tiny Executor Configuration

Executor Memory and Cores per Executor: Considering having 1 core per executor,
* Number of executors per node=8,
* Executor-memory=32/8=4GB
Calculating the Number of Executors: To calculate the number of executors, divide the available memory by the executor memory:
* Total memory available for Spark = 80% of 512 GB = 410 GB
* Number of executors = Total memory available for Spark / Executor memory = 410 GB / 4 GB ≈ 102 executors
* Number of executors per node = Total Number of Executors/ Number of Nodes = 102/16 ≈ 6 Executors/Node

- So, in this example, you would configure Spark with 102 executors, each executor having 1 core and 4 GB of memory.

####Fat Executor Configuration


The other way of configuring Spark Executor and its core is setting the maximum utility configuration i.e. having only one Executor per node and optimizing it based on the application performance.

Executor Memory and Cores per Executor: Considering having 8 cores per executor,
* Number of executors per node= number of cores for a node/ number of cores for an executor = 8/8 = 1,
* Executor-memory=32/1= 32GB
Calculating the Number of Executors: To calculate the number of executors, divide the available memory by the executor memory:
* Total memory available for Spark = 80% of 512 GB = 410 GB
* Number of executors = Total memory available for Spark / Executor memory = 410 GB / 32 GB ≈ 12 executors
* Number of executors per node = Total Number of Executors/ Number of Nodes = 12/16 ≈ 1 Executors/Node


**So, in this example, we would configure Spark with 16 executors, each executor having 8 core and 32 GB of memory.####**


####Balanced Executor Configuration

 **Spark founder Databricks after several trail and error testing the spark Executor and cores configuration, they recommends to have 2-5 cores per executor as the best initial efficient configuration for running the application smoothly.**

Executor Memory and Cores per Executor: Considering having 3 cores per executor, Leaving 1 core per node for daemon processes
* Number of executors per node= (number of cores for a node – core for daemon process)/ number of cores for an executor = 7/3 ≈ 2,
* Executor-memory=Total memory per node/ number executors per node = 32/2= 16GB
Calculating the Number of Executors: To calculate the number of executors, divide the available memory by the executor memory:
* Total memory available for Spark = 80% of 512 GB = 410 GB
* Number of executors = Total memory available for Spark / Executor memory = 410 GB / 16 GB ≈ 32 executors
* Number of executors per node = Total Number of Executors/ Number of Nodes = 32/16 = 2 Executors/Node

**Tiny Executor Configuration:**

- Pros: Resource efficiency, increased task isolation, better granularity for small tasks, and suitability for limited resources or a high number of concurrent tasks.
- Cons: Increased overhead, limited parallelism, potential bottlenecks, and memory overhead.

**Fat Executor Configuration:**
- Pros: Increased parallelism, reduced overhead, enhanced data locality, and improved performance for complex tasks.
- Cons: Resource overallocation, reduced task isolation, longer startup times, and potential challenges in resource sharing.

**Balanced Executor Configuration:**
- Pros: Optimal resource utilization, reasonable parallelism, flexibility for multiple workloads, and reduced overhead.
- Cons: Limited scaling, trade-off in task isolation, potential task granularity issues, and complexity in resource management.


#####Q1 How to apply row_number on a dataframe column?
#####Q2 Create a custom schema having columns as salary, firstname and lastname
#####Q3 cast a column into timestamp
#####Q4 how to drop null/how to handle nulls/how to drop na values
#####Q5 I have multiline in one column of my dataframe, how to achiev the right values or mul;tiple line in one single column 

In [0]:
############df=df.withColumn("Salried_row_num", row_number().over(Window.orderBy("Salary")))

2024-23-12 12:12:45

In [0]:
schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()

In [0]:
from pyspark.sql.functions import to_timestamp

(sc
    .parallelize([Row(dt='2016_08_21 11_31_08')])
    .toDF()
    .withColumn("parsed", to_timestamp("dt", "YYYY-MM-DD HH:mm:ss"))
    .show(1, False))

#####How can you cache data in PySpark to improve performance?

e methods .cache() or .persist() allows us to store the data in memory or at the specified storage level. This task improves performance by avoiding repeated computations and reducing the need for data serialization and deserialization. 

# How to cache data in memory 
df_from_csv.cache()

# How to persist data in local disk 
df_from_csv.persist(storageLevel=StorageLevel.DISK_ONLY)

#####What are the key differences between RDDs, DataFrames, and Datasets in PySpark?
1. Spark Resilient Distributed Datasets (RDD), DataFrame, and Datasets are key abstractions in Spark that enable us to work with structured data in a distributed computing environment. Even though they are all ways of representing data, they have key differences:

2. RDDs are low-level APIs that lack a schema and offer control over the data. They are immutable collections of objects 
DataFrames are high-level APIs built on top of RDDs optimized for performance but are not safe-type. They organize structured and semi-structured data into named columns.
3. Datasets combine the benefits of RDDs and DataFrames. They are high-level APIs that provide safe-type abstraction. They support Python and Scala and provide compile-time type checking while being faster than DataFrames.


#####What is the purpose of checkpoints in PySpark?
In PySpark, checkpointing implies that RDDs are saved to disk so this intermediate point can be referenced in the future instead of recomputing the RDD for the original source. Checkpoints provide a way to recover from failures because the driver is restarted with this previously computed state. 


#####What challenges have you faced when working with large datasets in PySpark? How did you overcome them?
With this question, we can relate to our own experience and tell a particular case in which encountered challenges with PySpark and large datasets that can include some of the following:

- Memory management and resource utilization.
- Data skewness and uneven workload distribution.
- Performance optimization, especially for wide transformations and shuffles.
- Debugging and troubleshooting complex job failures.
- Efficient data partitioning and storage
.
To overcome these issues, PySpark provides partitioning of the dataset, caching intermediate results, using built-in optimization techniques, robust cluster management, and leveraging fault tolerance mechanisms.